<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/Regression/Boston_housing_LinearRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output 
import tensorflow as tf
import pandas as pd
import tensorflow.feature_column as fc
from tensorflow.keras.datasets import boston_housing

In [100]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()
mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std
x_test -= mean
x_test /= std

#범죄율, 주거지역 비율, 비상업 지역 비율, 찰스강 인접여부, 산화질소 농도, 평균 방의 개수, 자가 소유비율, 가중거리, 순환 고속도로거리, 재산세율, 학생대 교사비율, 흑인 비율, 하위 계층 비율
#자가 가구의 중앙 값
features = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
x_train_df = pd.DataFrame(x_train, columns=features)
x_test_df = pd.DataFrame(x_test, columns=features)
y_train_df = pd.DataFrame(y_train, columns=['MEDV'])
y_test_df = pd.DataFrame(y_test, columns=['MEDV'])
x_train_df.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,-0.272246,-0.483615,-0.435762,-0.256833,-0.165227,-0.176443,0.813062,0.116698,-0.626249,-0.595170,1.148500,0.448077,0.825220
1,-0.403427,2.991784,-1.333912,-0.256833,-1.215182,1.894346,-1.910361,1.247585,-0.856463,-0.348433,-1.718189,0.431906,-1.329202
2,0.124940,-0.483615,1.028326,-0.256833,0.628642,-1.829688,1.110488,-1.187439,1.675886,1.565287,0.784476,0.220617,-1.308500
3,-0.401494,-0.483615,-0.869402,-0.256833,-0.361560,-0.324558,-1.236672,1.107180,-0.511142,-1.094663,0.784476,0.448077,-0.652926
4,-0.005634,-0.483615,1.028326,-0.256833,1.328612,0.153642,0.694808,-0.578572,1.675886,1.565287,0.784476,0.389882,0.263497


In [101]:
#특징 열 객체리스트 생성
feature_columns = []
for feature_name in features:
  feature_columns.append(fc.numeric_column(feature_name, dtype=tf.float32))
print(feature_columns)

[NumericColumn(key='CRIM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='ZN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='INDUS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='CHAS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='NOX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='AGE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='DIS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RAD', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='TAX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PTRATIO', shape=(1

In [104]:
#data set 객체를 feature 과 레이블을 배치로한 튜플을 만듬.
def estimator_input_fn(df_data, df_label, epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(df_data), df_label))
    if shuffle:
      ds = ds.shuffle(100)
    ds = ds.batch(batch_size).repeat(epochs)
    return ds
  return input_function

train_input_fn = estimator_input_fn(x_train_df, y_train_df)
val_input_fn = estimator_input_fn(x_test_df, y_test_df, epochs=1, shuffle=False)
print(train_input_fn)

<function estimator_input_fn.<locals>.input_function at 0x7f8b87da5c20>


In [106]:
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(val_input_fn)
result = linear_est.predict(val_input_fn)
clear_output()
for pred, exp in zip(result, y_test[:32]):
  print("Predicted value :", pred['predictions'][0], "expected : ", exp)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpl8n0l4m2/model.ckpt-130
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predicted value : -6.67315 expected :  7.2
Predicted value : 3.0489113 expected :  18.8
Predicted value : 4.288891 expected :  19.0
Predicted value : 10.53796 expected :  27.0
Predicted value : 6.7189727 expected :  22.2
Predicted value : 3.477635 expected :  24.5
Predicted value : 10.654866 expected :  31.2
Predicted value : 6.841366 expected :  22.9
Predicted value : 0.97139955 expected :  20.5
Predicted value : 1.7582672 expected :  23.2
Predicted value : 2.0056183 expected :  18.6
Predicted value : 1.7537775 expected :  14.5
Predicted value : -0.43305922 expected :  17.8
Predicted value : 13.322069 expected :  50.0
Predicted value : -1.7063828 expected :  20.8
Predicted value : 5.0059986 expected :  24.3
Predicted value